In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0,1,2,3'

In [2]:
%run -p ../train_cnf_disentangle_rl_2cond.py --data colormnist --dims 64,64,64 --strides 1,1,1,1 --num_blocks 2 --layer_type concat --multiscale True --rademacher True --batch_size 900 --test_batch_size 500 --save ../experiments_published/cnf_conditional_disentangle_colormnist_bs900_sratio_0_25_drop_0_5_rl_stdscale_6_2cond_nodropout_run1 --seed 1 --lr 0.001 --conditional True --controlled_tol False --train_mode semisup --log_freq 10 --weight_y 0.5 --condition_ratio 0.25 --dropout_rate 0.0 --scale_fac 1.0 --scale_std 6.0
#

/tancode/repos/tan-ffjord/train_cnf_disentangle_rl_2cond.py
from __future__ import print_function

import argparse
import os
import time
import numpy as np

import torch
import torch.optim as optim
import torchvision.datasets as dset
import torchvision.transforms as tforms
from torchvision.utils import save_image

import torch.utils.data as data
from torch.utils.data import Dataset

from PIL import Image
import os.path
import errno
import codecs

import lib.layers as layers
import lib.utils as utils
import lib.multiscale_parallel as multiscale_parallel
import lib.modules as modules
import lib.thops as thops

from train_misc import standard_normal_logprob
from train_misc import set_cnf_options, count_nfe, count_parameters, count_total_time, count_nfe_gate
from train_misc import add_spectral_norm, spectral_norm_power_iteration
from train_misc import create_regularization_fns, get_regularization, append_regularization_to_log

from tensorboardX import SummaryWriter

# go fast boi!!
torch.b

AssertionError: underflow in dt 0.0

         160801870 function calls (150161673 primitive calls) in 9632.271 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      924 5701.215    6.170 5701.215    6.170 {method 'run_backward' of 'torch._C._EngineBase' objects}
    41421 3357.130    0.081 3357.130    0.081 {method 'acquire' of '_thread.lock' objects}
   971600   53.236    0.000  316.902    0.000 train_cnf_disentangle_rl_2cond.py:214(__getitem__)
   971600   38.736    0.000   63.760    0.000 train_cnf_disentangle_rl_2cond.py:316(add_noise)
    27540   34.118    0.001   34.118    0.001 {method '_write_file' of 'torch._C.CudaFloatStorageBase' objects}
   971600   23.880    0.000  104.564    0.000 functional.py:32(to_tensor)
      331   23.257    0.070   23.257    0.070 {built-in method io.open}
      142   22.019    0.155   22.019    0.155 {built-in method posix.mkdir}
   971748   19.060    0.000   19.060    0.000 {method 'uniform_' of 'torch._C._TensorBase' obj